In [ ]:
%load_ext autoreload
%autoreload 2
import torch
import monai
from monai.transforms import (
    MapTransform,
    Compose,
    LoadImaged,
    SaveImaged,
    EnsureChannelFirstd,
    EnsureTyped,
    Spacingd,
    Orientationd,
    KeepLargestConnectedComponentd,
    FillHolesd,
    KeepLargestConnectedComponent, 
    FillHoles
)
from monai.data import decollate_batch
import numpy as np
import os
import matplotlib.pyplot as plt
import sys
repo_path = "/home/jovyan/work/repository/CT2MR/Generation_clean/"
sys.path.append(repo_path)
os.chdir(repo_path)
from data.solid_head_transform import GenerateSolidHeadMaskd,ApplyMaskAsBackgroundd


/opt/conda/envs/monai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def visualize(data_dict, slice_idx=None):
    """Visualizza una slice centrale se slice_idx non è specificato."""
    ct_masked = data_dict["ct"].cpu().numpy().squeeze()
    mr_masked = data_dict["mr"].cpu().numpy().squeeze()
    mask = data_dict["head_mask"].cpu().numpy().squeeze()

    if slice_idx is None:
        slice_idx = ct_masked.shape[2] // 2

    print(f"Visualizzazione della slice: {slice_idx}")

    plt.figure("Risultati MONAI", figsize=(18, 6))
    
    plt.subplot(1, 3, 1)
    plt.title("CT Mascherata")
    plt.imshow(ct_masked[:, :, slice_idx], cmap="gray")
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.title("MR Mascherata")
    plt.imshow(mr_masked[:, :, slice_idx], cmap="gray")
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.title("Maschera Finale Solida")
    plt.imshow(mask[:, :, slice_idx], cmap="gray")
    plt.axis('off')

    plt.show()

In [ ]:



output_dir = "treansform_test_output"


ct_path = "/home/jovyan/work/data/CTMR/DATASET/FDGMR-FTD/sub-95229/processed/CT/CTonMR/sub-95229ctMRspace.nii.gz"
mr_path = "/home/jovyan/work/data/CTMR/DATASET/FDGMR-FTD/sub-95229/sub-95229_t1w.nii.gz"

if not os.path.exists(ct_path) or not os.path.exists(mr_path):
    print("Errore: Percorsi dei file non validi.")
    print(f"Verifica che '{ct_path}' e '{mr_path}' esistano.")
    exit()

files = [{"ct": ct_path, "mr": mr_path}]


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(f"Utilizzo del device: {device}")

transforms = Compose([
    LoadImaged(keys=["ct", "mr"], reader="ITKReader"),
    EnsureChannelFirstd(keys=["ct", "mr"]),
#    Spacingd(keys=["ct", "mr"], pixdim=(1.5, 1.5, 1.5), mode=("bilinear", "bilinear")),
#    Orientationd(keys=["ct", "mr"], axcodes="RAS"),
    EnsureTyped(keys=["ct", "mr"], device=device, track_meta=True),
    
    GenerateSolidHeadMaskd(keys=("ct", "mr"), new_key="head_mask"),
    ApplyMaskAsBackgroundd(
            keys=["ct", "mr"], 
            mask_key="head_mask", 
            background_values={"ct": -1024, "mr": 0}
        ),
    
    SaveImaged(keys="head_mask", output_dir=output_dir, output_postfix="_solid_mask", resample=False, print_log=True),
    SaveImaged(keys="ct", output_dir=output_dir, output_postfix="_ct_masked", resample=False, print_log=True),
    SaveImaged(keys="mr", output_dir=output_dir, output_postfix="_mr_masked", resample=False, print_log=True),
])

print("\nInizio del processo di trasformazione...")
result_dict = transforms(files[0])
print("Processo completato con successo!")

visualize(result_dict)

## Applico la trasformazione ai dati

Molto pesante se non lo faccio prima, quindi preferisco preprocessare i dati

In [2]:
import os
import sys
repo_path = "/home/jovyan/work/repository/CT2MR/Generation_clean/"
sys.path.append(repo_path)
os.chdir(repo_path)

from data.datautils import MR2CTfindImages
import yaml
from utils import util

In [3]:
with open('experiments/MR2CT/options_1.yaml', 'r') as file:
    options = yaml.safe_load(file)


opt = util.DotDict(options)
dataset_option = util.DotDict(options['data'])
training_option = util.DotDict(options["training"])

In [4]:
train_image_paths, val_image_paths = MR2CTfindImages(dataset_option.basic_information,False)
test_image_paths = MR2CTfindImages(dataset_option.basic_information,True)

Dataset selezionati: ['OASIS3', 'AMY-MR', 'AMYFDGMR-MOV', 'Ginevra', 'MRADV-PT', 'FDGMR-FTD']
Elaborando dataset: OASIS3
Elaborando dataset: AMY-MR
Elaborando dataset: AMYFDGMR-MOV
Elaborando dataset: Ginevra
Elaborando dataset: MRADV-PT
Elaborando dataset: FDGMR-FTD
Dataset selezionati: ['RF', 'SynthRAD2023']
Elaborando dataset: RF
Elaborando dataset: SynthRAD2023


In [5]:
from data.solid_head_transform import GenerateSolidHeadMaskd,ApplyMaskAsBackgroundd


device = torch.device("cuda:0")
transforms = Compose([
    LoadImaged(keys=["CT", "MR"], reader="ITKReader",image_only = False),
    EnsureChannelFirstd(keys=["CT", "MR"]),
    # Spacingd(keys=["CT", "MR"], pixdim=(1.5, 1.5, 1.5), mode=("bilinear", "bilinear")),
    # Orientationd(keys=["CT", "MR"], axcodes="RAS"),
    EnsureTyped(keys=["CT", "MR"], device=device, track_meta=True),
    
    GenerateSolidHeadMaskd(keys=("CT", "MR"), new_key="head_mask"),
    ApplyMaskAsBackgroundd(
            keys=["CT", "MR"], 
            mask_key="head_mask", 
            background_values={"CT": -1024, "MR": 0}
        ),
])



In [6]:
image_paths = train_image_paths + val_image_paths + test_image_paths

In [7]:
batch_size = 1

In [8]:
from monai.data import Dataset,DataLoader,NibabelWriter
dataset = Dataset(
    data=image_paths,
    transform=transforms,
)
dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=False,
)




In [9]:
batch_data = next(iter(dataloader))

In [10]:
writer = NibabelWriter()

In [ ]:
from tqdm import tqdm
for batch_data in tqdm(dataloader):
    for i in range(batch_size):
        ct_tensor = batch_data["CT"][i]       
        mr_tensor = batch_data["MR"][i]

        ct_meta = {key: val[i] for key, val in batch_data["CT_meta_dict"].items()}
        mr_meta = {key: val[i] for key, val in batch_data["MR_meta_dict"].items()}


        ct_name = os.path.join(os.path.dirname(ct_meta["filename_or_obj"]),os.path.basename(ct_meta["filename_or_obj"]).replace("ctMRspace.nii.gz","ctMRspaceMRCTmasked.nii.gz"))
        mr_name = os.path.join(os.path.dirname(mr_meta["filename_or_obj"]),"processed","MR",os.path.basename(mr_meta["filename_or_obj"]).replace("_t1w.nii.gz","mrMRCTmasked.nii.gz"))

        # Salva il tensore CT processato
        writer.set_data_array(ct_tensor, channel_dim=0)
        writer.set_metadata(ct_meta)
        writer.write(ct_name, verbose=False)

        # Salva il tensore MR processato
        writer.set_data_array(mr_tensor, channel_dim=0)
        writer.set_metadata(mr_meta)
        writer.write(mr_name, verbose=False)

  0%|          | 10/2085 [00:50<2:51:18,  4.95s/it]

 60%|██████    | 1258/2090 [2:14:28<40:06,  2.89s/it]   